## Homework

### PFound
Исходные данные - Yandex Cup 2022 Analytics
- Ссылка - https://yandex.ru/cup/analytics/analysis/ , пример A. Рассчитать pFound
- Данные - https://yadi.sk/d/guqki4UI4hFlXQ
- Формула
$$pFound@K = \sum_{i=1}^{k} pLook[i]\ pRel[i]$$

$$pLook[1] = 1$$

$$pLook[i] = pLook[i-1]\ (1 - pRel[i-1])\ (1 - pBreak)$$

$$pBreak = 0.15$$

**Задача** - написать функцию, которая принимает на вход dataframe (после join), а на выходе дает средний pFound по всем query.
- Запрещается использовать циклы for для расчет метрики (как полностью, так и ее частей).
- Усложнение, если задача показалась легкой - попробуйте обойтись без groupby (не уверен, что это возможно, но вдруг вы справитесь)

In [1]:
import warnings

import numpy as np
import pandas as pd

In [2]:
!mkdir ../data

In [3]:
import requests
from urllib.parse import urlencode

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://yadi.sk/d/guqki4UI4hFlXQ'

# Получаем загрузочную ссылку
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

# Загружаем файл и сохраняем его
download_response = requests.get(download_url)
with open('../data/data.zip', 'wb') as f:   # Здесь укажите нужный путь к файлу
    f.write(download_response.content)

In [4]:
!unzip ../data/data.zip -d ../data
!rm ../data/hidden_task.zip
!unzip ../data/open_task.zip -d ../data
!rm ../data/open_task.zip
!rm ../data/data.zip

Archive:  ../data/data.zip
  inflating: ../data/hidden_task.zip  
  inflating: ../data/open_task.zip   
Archive:  ../data/open_task.zip
   creating: ../data/open_task/
  inflating: ../data/open_task/qid_query.tsv  
  inflating: ../data/open_task/hostid_url.tsv  
  inflating: ../data/open_task/qid_url_rating.tsv  


## 1. Решение с https://yandex.ru/cup/analytics/analysis/

In [5]:
# считываем данные
qid_query = pd.read_csv("../data/open_task/qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv("../data/open_task/qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv("../data/open_task/hostid_url.tsv", sep="\t", names=["hostid", "url"])

# делаем join двух таблиц, чтобы было просто брать url с максимальным рейтингом
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")


def plook(ind, rels):
    if ind == 0:
        return 1
    return plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)


def pfound(group):
    max_by_host = group.groupby("hostid")["rating"].max() # максимальный рейтинг хоста
    top10 = max_by_host.sort_values(ascending=False)[:10] # берем топ10 урлов с наивысшим рейтингом
    pfound = 0
    for ind, val in enumerate(top10):
        pfound += val*plook(ind, top10.values)
    return pfound

with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound) # группируем по qid и вычисляем pfound

qid_max = qid_pfound.idxmax() # берем qid с максимальным pfound
qid_query[qid_query["qid"] == qid_max]

,qid,query
12,295761,гугл переводчик


In [6]:
avg_pfound_original = qid_pfound.mean()
print(f'Средний pFound по всем query (расчет с сайта): {avg_pfound_original}')

Средний pFound по всем query (расчет с сайта): 0.4603173929969002


In [7]:
qid_pfound

qid
10387     0.497771
20860     0.655448
21070     0.497771
35618     0.437794
107538    0.354808
150126    0.366109
168170    0.481255
176370    0.393661
192007    0.191170
213932    0.347005
221830    0.497771
242953    0.497771
253476    0.497771
295761    0.900836
346214    0.263596
347852    0.618534
360100    0.470204
366042    0.309314
375608    0.497771
380923    0.429989
dtype: float64

## 2. Наше решение

In [8]:
qid_url_rating_hostid.head()

,qid,url,rating,hostid
0,10387,http://batman-arkhamcity.ru/,0.00,64
1,10387,http://bigtorrents.org/publ/batman_arkham_city...,0.14,71
2,10387,http://consolelife.ru/xbox-360/6577-o-rossiysk...,0.14,101
3,10387,http://dic.academic.ru/book.nsf/3662736/Batman...,0.00,115
4,10387,http://forum.csmania.ru/viewtopic.php?t=25986,0.14,155


In [9]:
''' Если для запроса есть несколько документов с одним и тем же id хоста — оставить
    только максимально релевантный документ (а если несколько документов максимально релевантны, выбрать любой).
'''
qid_hostid_max_rel = qid_url_rating_hostid.groupby(['qid', 'hostid'])['rating'].max().reset_index()
qid_hostid_max_rel

,qid,hostid,rating
0,10387,64,0.00
1,10387,71,0.14
2,10387,101,0.14
3,10387,115,0.00
4,10387,155,0.14
...,...,...,...
674,380923,1063,0.00
675,380923,1065,0.00
676,380923,1105,0.00
677,380923,1168,0.00


In [10]:
''' Для каждого запроса получим топ 10 (или меньше, если нет 10) hostid с рейтингами начиная
    с максимального и упорядоченными про убыванию
'''

qid_top_10_hostid = qid_hostid_max_rel.sort_values(['qid', 'rating'], ascending=False).groupby(['qid']).head(10)
qid_top_10_hostid.head(20)

,qid,hostid,rating
647,380923,6,0.14
654,380923,179,0.14
662,380923,386,0.14
666,380923,484,0.14
665,380923,462,0.07
668,380923,670,0.07
673,380923,1028,0.07
678,380923,1172,0.07
648,380923,45,0.00
649,380923,67,0.00


In [11]:
# Порядковые номера для hostid внутри каждого qid (0-9)
qid_top_10_hostid['n'] = qid_top_10_hostid.groupby('qid').cumcount()

# Рассчитаем вспомогательные множители для расчета plook: в формуле это (1 - Rel) и (1 - pBreak)
qid_top_10_hostid['one_minus_rating'] = (1 - qid_top_10_hostid['rating']).shift(1)
qid_top_10_hostid['one_minus_p_break'] = 1 - 0.15
qid_top_10_hostid.loc[qid_top_10_hostid['n'] == 0, ['one_minus_p_break', 'one_minus_rating']] = 1

# Расчитаем вспомогательную колонку, из которой затем просто используя cumprod получим plool
qid_top_10_hostid['multiplier'] = qid_top_10_hostid['one_minus_rating'] * qid_top_10_hostid['one_minus_p_break']

qid_top_10_hostid['plook'] = qid_top_10_hostid.groupby('qid')['multiplier'].cumprod()
qid_top_10_hostid['pfound'] = qid_top_10_hostid['plook'] * qid_top_10_hostid['rating']

qid_top_10_hostid.head(20)

,qid,hostid,rating,n,one_minus_rating,one_minus_p_break,multiplier,plook,pfound
647,380923,6,0.14,0,1.00,1.00,1.0000,1.000000,0.140000
654,380923,179,0.14,1,0.86,0.85,0.7310,0.731000,0.102340
662,380923,386,0.14,2,0.86,0.85,0.7310,0.534361,0.074811
666,380923,484,0.14,3,0.86,0.85,0.7310,0.390618,0.054687
665,380923,462,0.07,4,0.86,0.85,0.7310,0.285542,0.019988
668,380923,670,0.07,5,0.93,0.85,0.7905,0.225721,0.015800
673,380923,1028,0.07,6,0.93,0.85,0.7905,0.178432,0.012490
678,380923,1172,0.07,7,0.93,0.85,0.7905,0.141051,0.009874
648,380923,45,0.00,8,0.93,0.85,0.7905,0.111501,0.000000
649,380923,67,0.00,9,1.00,0.85,0.8500,0.094775,0.000000


In [12]:
pfound_qid = qid_top_10_hostid.groupby('qid')['pfound'].sum()
print(f'PFound metric for qid: ')
pfound_qid

PFound metric for qid: 


qid
10387     0.497771
20860     0.655448
21070     0.497771
35618     0.437794
107538    0.354808
150126    0.366109
168170    0.481255
176370    0.393661
192007    0.191170
213932    0.347005
221830    0.497771
242953    0.497771
253476    0.497771
295761    0.900836
346214    0.263596
347852    0.618534
360100    0.470204
366042    0.309314
375608    0.497771
380923    0.429989
Name: pfound, dtype: float64

## 3. Оформим решение в виде функции

In [17]:
def mean_PFoundK(df: pd.DataFrame, K: int = 10, p_break: float = 0.15) -> float:
    ''' Calculate average PFound@K '''
    qid_hostid_max_rel = df.groupby(['qid', 'hostid'])['rating'].max().reset_index()
    qid_top_k_hostid = qid_hostid_max_rel.sort_values(['qid', 'rating'], ascending=False).groupby(['qid']).head(K)
    # Порядковые номера для hostid внутри каждого qid (0: k - 1)
    qid_top_k_hostid['n'] = qid_top_k_hostid.groupby('qid').cumcount()

    # Рассчитаем вспомогательные множители для расчета plook: в формуле это (1 - Rel) и (1 - pBreak)
    qid_top_k_hostid['one_minus_rating'] = (1 - qid_top_k_hostid['rating']).shift(1)
    qid_top_k_hostid['one_minus_p_break'] = 1 - p_break
    qid_top_k_hostid.loc[qid_top_10_hostid['n'] == 0, ['one_minus_p_break', 'one_minus_rating']] = 1

    # Расчитаем вспомогательную колонку, из которой затем просто используя cumprod получим plool
    qid_top_k_hostid['multiplier'] = qid_top_k_hostid['one_minus_rating'] * qid_top_k_hostid['one_minus_p_break']

    qid_top_k_hostid['plook'] = qid_top_k_hostid.groupby('qid')['multiplier'].cumprod()
    qid_top_k_hostid['pfound'] = qid_top_k_hostid['plook'] * qid_top_k_hostid['rating']
    
    pfound_qid = qid_top_k_hostid.groupby('qid')['pfound'].sum()
    return pfound_qid.mean()

In [18]:
avg_pfound = mean_PFound(qid_url_rating_hostid)
print(f'Средний PFound по всем query: {avg_pfound}')
assert np.abs(avg_pfound - avg_pfound_original) < 1e-6, 'PFound metrics is not equal'

Средний PFound по всем query: 0.4603173929969002
